# Self-Driving Car Engineer Nanodegree

## Deep Learning

## Project: Build a Traffic Sign Recognition Classifier

In this notebook, a template is provided for you to implement your functionality in stages which is required to successfully complete this project. If additional code is required that cannot be included in the notebook, be sure that the Python code is successfully imported and included in your submission, if necessary. Sections that begin with **'Implementation'** in the header indicate where you should begin your implementation for your project. Note that some sections of implementation are optional, and will be marked with **'Optional'** in the header.

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

---
## Step 0: Load The Data

In [1]:
# Load pickled data
import pickle

# TODO: Fill this in based on where you saved the training and testing data

training_file = './traffic-signs-data/train.p'
testing_file = './traffic-signs-data/test.p'

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
X_train_full, y_train_full = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

Vendor:  Continuum Analytics, Inc.
Package: mkl
Message: trial mode expires in 30 days


In [2]:
from sklearn.cross_validation import train_test_split

X_train, X_validation, y_train, y_validation = train_test_split(X_train_full, y_train_full, test_size=0.20, random_state=42)


In [3]:
print("len(X_train))", len(X_train))
print("len(X_validation))", len(X_validation))
print("len(y_train))", len(y_train))
print("len(y_validation))", len(y_validation))

len(X_train)) 31367
len(X_validation)) 7842
len(y_train)) 31367
len(y_validation)) 7842


---

## Step 1: Dataset Summary & Exploration

The pickled data is a dictionary with 4 key/value pairs:

- `'features'` is a 4D array containing raw pixel data of the traffic sign images, (num examples, width, height, channels).
- `'labels'` is a 2D array containing the label/class id of the traffic sign. The file `signnames.csv` contains id -> name mappings for each id.
- `'sizes'` is a list containing tuples, (width, height) representing the the original width and height the image.
- `'coords'` is a list containing tuples, (x1, y1, x2, y2) representing coordinates of a bounding box around the sign in the image. **THESE COORDINATES ASSUME THE ORIGINAL IMAGE. THE PICKLED DATA CONTAINS RESIZED VERSIONS (32 by 32) OF THESE IMAGES**

Complete the basic data summary below.

In [4]:
import numpy as np

### Replace each question mark with the appropriate value.

# TODO: Number of training examples
n_train = len(X_train)

# TODO: Number of testing examples.
n_test = len(X_test)

# TODO: What's the shape of an traffic sign image?
image_shape = np.array(X_train[0]).shape

# TODO: How many unique classes/labels there are in the dataset.
n_classes = len(set(y_train))

print("Number of training examples =", n_train)
print("Number of testing examples =", n_test)
print("Image data shape =", image_shape)
print("Number of classes =", n_classes)

Number of training examples = 31367
Number of testing examples = 12630
Image data shape = (32, 32, 3)
Number of classes = 43


Visualize the German Traffic Signs Dataset using the pickled file(s). This is open ended, suggestions include: plotting traffic sign images, plotting the count of each sign, etc.

The [Matplotlib](http://matplotlib.org/) [examples](http://matplotlib.org/examples/index.html) and [gallery](http://matplotlib.org/gallery.html) pages are a great resource for doing visualizations in Python.

**NOTE:** It's recommended you start with something simple first. If you wish to do more, come back to it after you've completed the rest of the sections.

In [5]:
### Data exploration visualization goes here.
### Feel free to use as many code cells as needed.
#import matplotlib.pyplot as plt
# Visualizations will be shown in the notebook.
#%matplotlib inline

#import matplotlib.image as mpimg
#i=13101
#print(y_train[i])
#plt.imshow(X_train[i])
    

----

## Step 2: Design and Test a Model Architecture

Design and implement a deep learning model that learns to recognize traffic signs. Train and test your model on the [German Traffic Sign Dataset](http://benchmark.ini.rub.de/?section=gtsrb&subsection=dataset).

There are various aspects to consider when thinking about this problem:

- Neural network architecture
- Play around preprocessing techniques (normalization, rgb to grayscale, etc)
- Number of examples per label (some have more than others).
- Generate fake data.

Here is an example of a [published baseline model on this problem](http://yann.lecun.com/exdb/publis/pdf/sermanet-ijcnn-11.pdf). It's not required to be familiar with the approach used in the paper but, it's good practice to try to read papers like these.

**NOTE:** The LeNet-5 implementation shown in the [classroom](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) at the end of the CNN lesson is a solid starting point. You'll have to change the number of classes and possibly the preprocessing, but aside from that it's plug and play!

### Implementation

Use the code cell (or multiple code cells, if necessary) to implement the first step of your project. Once you have completed your implementation and are satisfied with the results, be sure to thoroughly answer the questions that follow.

In [6]:
### Preprocess the data here.
### Feel free to use as many code cells as needed.

In [7]:
import tensorflow as tf

assert(len(X_train) == len(y_train))
assert(len(X_test) == len(y_test))

print()
print("Image Shape: {}".format(X_train[0].shape))
print()
print("Training Set:   {} samples".format(len(X_train)))
print("Test Set:       {} samples".format(len(X_test)))



Image Shape: (32, 32, 3)

Training Set:   31367 samples
Test Set:       12630 samples


In [8]:
import random
import numpy as np
#import matplotlib.pyplot as plt
#%matplotlib inline

#index = random.randint(0, len(X_train))
#image = X_train[index].squeeze()

#plt.figure(figsize=(1,1))
#plt.imshow(image)
#print(y_train[index])

In [9]:
from sklearn.utils import shuffle

X_train, y_train = shuffle(X_train, y_train)

In [10]:
import tensorflow as tf

EPOCHS = 50
BATCH_SIZE = 128


from tensorflow.contrib.layers import flatten

def conv2d(x,W,b, strides=1):
    
    #print(x)
    #print(strides)
    #print(W.eval())
    
    x = tf.nn.conv2d(x,W,strides=[1, strides, strides,1], padding='VALID')
    x = tf.nn.bias_add(x,b)
    return tf.nn.relu(x)

def maxpool2d(x,k=2):
    return tf.nn.max_pool( x, ksize = [1,k,k,1], strides = [1,k,k,1], padding='VALID')
    
def LeNet(x):    
    # Hyperparameters
    mu = 0
    sigma = 0.1
    
    weight_layer_1 = tf.Variable(tf.truncated_normal( shape=(5, 5, 3, 6), mean=mu, stddev=sigma ) )#tf.Variable(tf.truncated_normal( [5, 5, 1, 6]))
    bias_layer_1 = tf.Variable(tf.zeros(6))

    weight_layer_2 = tf.Variable(tf.truncated_normal(shape=(5,5,6,16), mean=mu,stddev=sigma) )
    bias_layer_2 = tf.Variable(tf.zeros(16))

    weight_connected = tf.Variable(tf.truncated_normal([5*5*16, 400]))
    bias_connected = tf.Variable(tf.zeros(400))

    weight_layer_3 = tf.Variable(tf.truncated_normal(shape=(400, 120),mean=mu,stddev=sigma))
    bias_layer_3 = tf.Variable(tf.zeros(120))

    weight_layer_4 = tf.Variable(tf.truncated_normal(shape=(120,84),mean=mu, stddev=sigma))
    bias_layer_4 = tf.Variable(tf.zeros(84))

    weight_layer_5 = tf.Variable(tf.truncated_normal(shape=(84,43), mean=mu, stddev=sigma))
    bias_layer_5 = tf.Variable(tf.zeros(43))    
    
    
    # TODO: Layer 1: Convolutional. Input = 32x32x1. Output = 28x28x6.
    conv1 = conv2d(x,weight_layer_1,bias_layer_1,1 )
    # TODO: Activation.
    # TODO: Pooling. Input = 28x28x6. Output = 14x14x6.
    conv1 = maxpool2d(conv1)

    # TODO: Layer 2: Convolutional. Output = 10x10x16.
    # TODO: Activation.
    # TODO: Pooling. Input = 10x10x16. Output = 5x5x16.
    conv2 = conv2d(conv1,weight_layer_2,bias_layer_2,1 )
    conv2 = maxpool2d(conv2)


    # TODO: Flatten. Input = 5x5x16. Output = 400.
    #fc1 = tf.reshape(conv2, 
    #                 [-1, weight_connected.get_shape().as_list()[0]])
    #fc1 = tf.add(tf.matmul(fc1, weight_connected), bias_connected)
    #fc1 = tf.nn.tanh(fc1)
    fc1 = flatten(conv2)
    
    # TODO: Layer 3: Fully Connected. Input = 400. Output = 120.
    conv3 = tf.add(tf.matmul(fc1,weight_layer_3), bias_layer_3)
    # TODO: Activation.
    conv3 = tf.nn.relu(conv3)
    
    # TODO: Layer 4: Fully Connected. Input = 120. Output = 84.
    conv4 = tf.add(tf.matmul(conv3,weight_layer_4), bias_layer_4)
    # TODO: Activation.
    conv4 = tf.nn.relu(conv4)

    # TODO: Layer 5: Fully Connected. Input = 84. Output = 10.
    logits = tf.add(tf.matmul(conv4,weight_layer_5), bias_layer_5)

    
    return logits



x = tf.placeholder(tf.float32, (None, 32, 32, 3))
y = tf.placeholder(tf.int32, (None))
one_hot_y = tf.one_hot(y, 43)


rate = 0.001

logits = LeNet(x)

sm = tf.nn.softmax(logits)
prediction = tf.argmax(sm, 1)


cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits, one_hot_y)
loss_operation = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate = rate)
training_operation = optimizer.minimize(loss_operation)

correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot_y, 1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

def evaluate(X_data, y_data):
    num_examples = len(X_data)
    total_accuracy = 0
    sess = tf.get_default_session()
    for offset in range(0, num_examples, BATCH_SIZE):
        batch_x, batch_y = X_data[offset:offset+BATCH_SIZE], y_data[offset:offset+BATCH_SIZE]
        accuracy = sess.run(accuracy_operation, feed_dict={x: batch_x, y: batch_y})
        total_accuracy += (accuracy * len(batch_x))
    return total_accuracy / num_examples


with tf.Session() as sess:
    #sess.run(tf.global_variables_initializer())
    sess.run(tf.initialize_all_variables())
    
    num_examples = len(X_train)
    
    print("Training...")
    print()
    for i in range(EPOCHS):
        X_train, y_train = shuffle(X_train, y_train)
        for offset in range(0, num_examples, BATCH_SIZE):
            end = offset + BATCH_SIZE
            batch_x, batch_y = X_train[offset:end], y_train[offset:end]
            sess.run(training_operation, feed_dict={x: batch_x, y: batch_y})
            
        validation_accuracy = evaluate(X_validation, y_validation)
        #validation_accuracy = evaluate(X_test, y_test)
        print("EPOCH {} ...".format(i+1))
        print("Validation Accuracy = {:.3f}".format(validation_accuracy))
        print()
        
    try:
        saver
    except NameError:
        saver = tf.train.Saver()
    saver.save(sess, 'lenet')
    print("Model saved")

Training...

EPOCH 1 ...
Validation Accuracy = 0.657

EPOCH 2 ...
Validation Accuracy = 0.841

EPOCH 3 ...
Validation Accuracy = 0.900

EPOCH 4 ...
Validation Accuracy = 0.926

EPOCH 5 ...
Validation Accuracy = 0.933

EPOCH 6 ...
Validation Accuracy = 0.937

EPOCH 7 ...
Validation Accuracy = 0.945

EPOCH 8 ...
Validation Accuracy = 0.943

EPOCH 9 ...
Validation Accuracy = 0.950

EPOCH 10 ...
Validation Accuracy = 0.941

EPOCH 11 ...
Validation Accuracy = 0.959

EPOCH 12 ...
Validation Accuracy = 0.966

EPOCH 13 ...
Validation Accuracy = 0.955

EPOCH 14 ...
Validation Accuracy = 0.958

EPOCH 15 ...
Validation Accuracy = 0.957

EPOCH 16 ...
Validation Accuracy = 0.953

EPOCH 17 ...
Validation Accuracy = 0.961

EPOCH 18 ...
Validation Accuracy = 0.976

EPOCH 19 ...
Validation Accuracy = 0.974

EPOCH 20 ...
Validation Accuracy = 0.973

EPOCH 21 ...
Validation Accuracy = 0.960

EPOCH 22 ...
Validation Accuracy = 0.972

EPOCH 23 ...
Validation Accuracy = 0.972

EPOCH 24 ...
Validation Accura

In [11]:
with tf.Session() as sess:
    sess.run(tf.initialize_all_variables()) #It will show error without this line
    loader = tf.train.import_meta_graph('lenet.meta')
    loader.restore(sess, tf.train.latest_checkpoint('./'))

    test_accuracy = evaluate(X_test, y_test)
    print("Test Accuracy = {:.3f}".format(test_accuracy))


Test Accuracy = 0.898


### Question 1 

_Describe how you preprocessed the data. Why did you choose that technique?_

**Answer:**

In [12]:
### Generate data additional data (OPTIONAL!)
### and split the data into training/validation/testing sets here.
### Feel free to use as many code cells as needed.

In [13]:
##Answer: Split the data into train and testing. Resuse the testing data as validation data.


### Question 2

_Describe how you set up the training, validation and testing data for your model. **Optional**: If you generated additional data, how did you generate the data? Why did you generate the data? What are the differences in the new dataset (with generated data) from the original dataset?_

**Answer:**

In [14]:
### Define your architecture here.
### Feel free to use as many code cells as needed.

###Layer 1: Convolutional. The output shape should be 28x28x6.
###Activation. Your choice of activation function.
###Pooling. The output shape should be 14x14x6.
###Layer 2: Convolutional. The output shape should be 10x10x16.
###Activation. Your choice of activation function.
###Pooling. The output shape should be 5x5x16.
###Flatten. Flatten the output shape of the final pooling layer such that it's 1D instead of 3D. 
###Activation. Your choice of activation function.
###Layer 4: Fully Connected. This should have 84 outputs.
###Activation. Your choice of activation function.
###Layer 5: Fully Connected (Logits). This should have 43 outputs.

### Question 3

_What does your final architecture look like? (Type of model, layers, sizes, connectivity, etc.)  For reference on how to build a deep neural network using TensorFlow, see [Deep Neural Network in TensorFlow
](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/b516a270-8600-4f93-a0a3-20dfeabe5da6/concepts/83a3a2a2-a9bd-4b7b-95b0-eb924ab14432) from the classroom._


**Answer:**

In [15]:
### Train your model here.
### Feel free to use as many code cells as needed.

    ###mu = 0
    ###sigma = 0.1
    
    ###weight_layer_1 = tf.Variable(tf.truncated_normal( shape=(5, 5, 3, 6), mean=mu, stddev=sigma ) )#tf.Variable(tf.truncated_normal( [5, 5, 1, 6]))
    ###bias_layer_1 = tf.Variable(tf.zeros(6))

    ###weight_layer_2 = tf.Variable(tf.truncated_normal(shape=(5,5,6,16), mean=mu,stddev=sigma) )
    ###bias_layer_2 = tf.Variable(tf.zeros(16))

    ###weight_connected = tf.Variable(tf.truncated_normal([5*5*16, 400]))
    ###bias_connected = tf.Variable(tf.zeros(400))

    ###weight_layer_3 = tf.Variable(tf.truncated_normal(shape=(400, 120),mean=mu,stddev=sigma))
    ###bias_layer_3 = tf.Variable(tf.zeros(120))

    ###weight_layer_4 = tf.Variable(tf.truncated_normal(shape=(120,84),mean=mu, stddev=sigma))
    ###bias_layer_4 = tf.Variable(tf.zeros(84))

    ###weight_layer_5 = tf.Variable(tf.truncated_normal(shape=(84,43), mean=mu, stddev=sigma))
    ###bias_layer_5 = tf.Variable(tf.zeros(43))    
    
    
    # Layer 1: Convolutional. Input = 32x32x3. Output = 28x28x6.
    ###conv1 = conv2d(x,weight_layer_1,bias_layer_1,1 )
    # Activation.
    # Pooling. Input = 28x28x6. Output = 14x14x6.
    ###conv1 = maxpool2d(conv1)

    # Layer 2: Convolutional. Output = 10x10x16.
    # Activation.
    # Pooling. Input = 10x10x16. Output = 5x5x16.
    ###conv2 = conv2d(conv1,weight_layer_2,bias_layer_2,1 )
    ###conv2 = maxpool2d(conv2)


    # Flatten. Input = 5x5x16. Output = 400.
    ###fc1 = flatten(conv2)
    
    # Layer 3: Fully Connected. Input = 400. Output = 120.
    ###conv3 = tf.add(tf.matmul(fc1,weight_layer_3), bias_layer_3)
    # Activation.
    ###conv3 = tf.nn.relu(conv3)
    
    # Layer 4: Fully Connected. Input = 120. Output = 84.
    ###conv4 = tf.add(tf.matmul(conv3,weight_layer_4), bias_layer_4)
    # Activation.
    ###conv4 = tf.nn.relu(conv4)

    #Layer 5: Fully Connected. Input = 84. Output = 10.
    ###logits = tf.add(tf.matmul(conv4,weight_layer_5), bias_layer_5)



### Question 4

_How did you train your model? (Type of optimizer, batch size, epochs, hyperparameters, etc.)_



**Answer:**

EPOCHS = 50

BATCH_SIZE = 128

rate = 0.001

cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits, one_hot_y)

loss_operation = tf.reduce_mean(cross_entropy)

optimizer = tf.train.AdamOptimizer(learning_rate = rate)

training_operation = optimizer.minimize(loss_operation)

correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot_y, 1))

accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

### Question 5


_What approach did you take in coming up with a solution to this problem? It may have been a process of trial and error, in which case, outline the steps you took to get to the final solution and why you chose those steps. Perhaps your solution involved an already well known implementation or architecture. In this case, discuss why you think this is suitable for the current problem._

**Answer:**

I use leNet to solve this problem because the traffic sign dataset and results are similiar with the letNet issue. 

---

## Step 3: Test a Model on New Images

Take several pictures of traffic signs that you find on the web or around you (at least five), and run them through your classifier on your computer to produce example results. The classifier might not recognize some local signs but it could prove interesting nonetheless.

You may find `signnames.csv` useful as it contains mappings from the class id (integer) to the actual sign name.

### Implementation

Use the code cell (or multiple code cells, if necessary) to implement the first step of your project. Once you have completed your implementation and are satisfied with the results, be sure to thoroughly answer the questions that follow.

In [16]:
### Load the images and plot them here.
### Feel free to use as many code cells as needed.
'''
with tf.Session() as sess:
    sess.run(tf.initialize_all_variables()) #It will show error without this line
    loader = tf.train.import_meta_graph('lenet.meta')
    loader.restore(sess, tf.train.latest_checkpoint('./'))

    test_accuracy = evaluate(X_test, y_test)
    print("Test Accuracy = {:.3f}".format(test_accuracy))
'''

'\nwith tf.Session() as sess:\n    sess.run(tf.initialize_all_variables()) #It will show error without this line\n    loader = tf.train.import_meta_graph(\'lenet.meta\')\n    loader.restore(sess, tf.train.latest_checkpoint(\'./\'))\n\n    test_accuracy = evaluate(X_test, y_test)\n    print("Test Accuracy = {:.3f}".format(test_accuracy))\n'

### Question 6

_Choose five candidate images of traffic signs and provide them in the report. Are there any particular qualities of the image(s) that might make classification difficult? It could be helpful to plot the images in the notebook._



**Answer:**

In [17]:
### Run the predictions here.
### Feel free to use as many code cells as needed.


predict_indexes = [0,3000,6000,9000,12000]


with tf.Session() as sess:
    sess.run(tf.initialize_all_variables()) #It will show error without this line
    loader = tf.train.import_meta_graph('lenet.meta')
    loader.restore(sess, tf.train.latest_checkpoint('./'))

    for idx in predict_indexes:
        predict_img = X_train[idx]
        train_y = y_train[idx]
        
        classi = sess.run(prediction, feed_dict={x: [predict_img]} )
        
        print("classification of index ", idx, "  = " , classi, " real value = ", train_y)

classification of index  0   =  [9]  real value =  9
classification of index  3000   =  [5]  real value =  3
classification of index  6000   =  [4]  real value =  4
classification of index  9000   =  [25]  real value =  25
classification of index  12000   =  [42]  real value =  42


### Question 7

_Is your model able to perform equally well on captured pictures when compared to testing on the dataset? The simplest way to do this check the accuracy of the predictions. For example, if the model predicted 1 out of 5 signs correctly, it's 20% accurate._

_**NOTE:** You could check the accuracy manually by using `signnames.csv` (same directory). This file has a mapping from the class id (0-42) to the corresponding sign name. So, you could take the class id the model outputs, lookup the name in `signnames.csv` and see if it matches the sign from the image._


**Answer:**

In [18]:
### Visualize the softmax probabilities here.
### Feel free to use as many code cells as needed.

'''
The accuracy of the result is 90%+
'''

'\nThe accuracy of the result is 90%+\n'

### Question 8

*Use the model's softmax probabilities to visualize the **certainty** of its predictions, [`tf.nn.top_k`](https://www.tensorflow.org/versions/r0.12/api_docs/python/nn.html#top_k) could prove helpful here. Which predictions is the model certain of? Uncertain? If the model was incorrect in its initial prediction, does the correct prediction appear in the top k? (k should be 5 at most)*

`tf.nn.top_k` will return the values and indices (class ids) of the top k predictions. So if k=3, for each sign, it'll return the 3 largest probabilities (out of a possible 43) and the correspoding class ids.

Take this numpy array as an example:

```
# (5, 6) array
a = np.array([[ 0.24879643,  0.07032244,  0.12641572,  0.34763842,  0.07893497,
         0.12789202],
       [ 0.28086119,  0.27569815,  0.08594638,  0.0178669 ,  0.18063401,
         0.15899337],
       [ 0.26076848,  0.23664738,  0.08020603,  0.07001922,  0.1134371 ,
         0.23892179],
       [ 0.11943333,  0.29198961,  0.02605103,  0.26234032,  0.1351348 ,
         0.16505091],
       [ 0.09561176,  0.34396535,  0.0643941 ,  0.16240774,  0.24206137,
         0.09155967]])
```

Running it through `sess.run(tf.nn.top_k(tf.constant(a), k=3))` produces:

```
TopKV2(values=array([[ 0.34763842,  0.24879643,  0.12789202],
       [ 0.28086119,  0.27569815,  0.18063401],
       [ 0.26076848,  0.23892179,  0.23664738],
       [ 0.29198961,  0.26234032,  0.16505091],
       [ 0.34396535,  0.24206137,  0.16240774]]), indices=array([[3, 0, 5],
       [0, 1, 4],
       [0, 5, 1],
       [1, 3, 5],
       [1, 4, 3]], dtype=int32))
```

Looking just at the first row we get `[ 0.34763842,  0.24879643,  0.12789202]`, you can confirm these are the 3 largest probabilities in `a`. You'll also notice `[3, 0, 5]` are the corresponding indices.

**Answer:**

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.

In [19]:
### Run the predictions here.
### Feel free to use as many code cells as needed.


predict_indexes = [0,3000,6000,9000,12000]


with tf.Session() as sess:
    sess.run(tf.initialize_all_variables()) #It will show error without this line
    loader = tf.train.import_meta_graph('lenet.meta')
    loader.restore(sess, tf.train.latest_checkpoint('./'))

    for idx in predict_indexes:
        predict_img = X_train[idx]
        train_y = y_train[idx]
        
        softmaxes = sess.run(tf.nn.top_k(sm, k=3), feed_dict={x: [predict_img]} )
        
        print(idx)
        print(softmaxes)
        
        #print("classification of index ", idx, "  = " , classi, " real value = ", train_y)

0
TopKV2(values=array([[ 1.,  0.,  0.]], dtype=float32), indices=array([[9, 0, 1]], dtype=int32))
3000
TopKV2(values=array([[  7.06583440e-01,   2.92896241e-01,   5.20367525e-04]], dtype=float32), indices=array([[5, 3, 2]], dtype=int32))
6000
TopKV2(values=array([[  1.00000000e+00,   1.28985309e-22,   5.03686126e-30]], dtype=float32), indices=array([[4, 1, 0]], dtype=int32))
9000
TopKV2(values=array([[  1.00000000e+00,   7.51623144e-19,   5.94960536e-23]], dtype=float32), indices=array([[25, 30, 31]], dtype=int32))
12000
TopKV2(values=array([[  1.00000000e+00,   2.00385934e-32,   1.78308362e-34]], dtype=float32), indices=array([[42, 40,  5]], dtype=int32))
